# 03 — Semantic Search & Reformulation

**Objective.**
- Embed FAQ entries, compute similarity against a user query.
- Use LLM router: prefer local Ollama if available, else Hugging Face fallback.
- Return top-k with confidence.

**Notes.** Keep everything lightweight and reproducible, no external secrets.


In [ ]:
import json, pathlib
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

ROOT = pathlib.Path(__file__).resolve().parents[1]
DATA = ROOT / "data"
faq = json.loads((DATA / "faq_dataset.json").read_text(encoding="utf-8"))

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
texts = [f"{x['question']} {x['answer']}" for x in faq]
emb = model.encode(texts, convert_to_numpy=True)

def topk(query: str, k: int = 3):
    qv = model.encode([query], convert_to_numpy=True)
    sims = cosine_similarity(qv, emb).ravel()
    idx = np.argsort(-sims)[:k]
    return [(faq[i], float(sims[i])) for i in idx]

topk("how to reserve a book?", k=3)